# Getting the list of movies with actors

In [21]:
import requests
import json

# Set API key and base URL for TMDB API
api_key = '51d8c99884d982fb01247c3fcc8e9c96'
base_url = 'https://api.themoviedb.org/3'

# Define function to get top American movies
def get_top_movies():
    movies = []
    page = 1
    while len(movies) < 365:
        url = f"{base_url}/discover/movie?api_key={api_key}&sort_by=vote_average.desc&with_original_language=en&region=US&page={page}&include_adult=false"
        response = requests.get(url)
        if response.status_code == 200:
            page_movies = response.json()['results']
            if len(page_movies) == 0:
                break
            for movie in page_movies:
                if movie['cast'] > 4:
                    movies += page_movies
            page += 1
        else:
            print(f"Error getting top movies: {response.text}")
            break
    return movies

# Define function to get first 5 actors for a movie
def get_movie_actors(movie_id):
    url = f"{base_url}/movie/{movie_id}/credits?api_key={api_key}&language=en-US"
    response = requests.get(url)
    if response.status_code == 200:
        actors = response.json()['cast'][:5]
        return [actor['name'] for actor in actors]
    else:
        print(f"Error getting movie credits for {movie_id}: {response.text}")
        return []

# Get top American movies
movies = get_top_movies()

# Save actors for each movie into separate JSON files
for movie in movies:
    movie_id = movie['id']
    title = movie['title']
    year = movie['release_date'][:4]
    actors = get_movie_actors(movie_id)
    filename = f"movies/{movie_id}_actors.json"
    with open(filename, 'w') as f:
        f.write(json.dumps({
            'title': title,
            'year': year,
            'actors': actors
        }, indent=4))


KeyError: 'cast'

## Cleanup

In [18]:
import os, shutil
folder = './movies'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))